In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import re
import time
import mysql.connector
import json
import requests





urls= ['https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?px=default&city=%E5%B9%BF%E5%B7%9E#filterBox',
      'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?px=default&city=%E6%B7%B1%E5%9C%B3#filterBox']

conn = mysql.connector.connect(user='name', password='', database='lagou')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE job_info (                                                                                                                          
                num             int             NOT NULL AUTO_INCREMENT    PRIMARY KEY,
                city            varchar(20)     NOT NULL,
                pos_name        varchar(50)     NOT NULL,                                                                                                                        
                company         varchar(50)     NOT NULL,                                                                                                                        
                experience      varchar(20)     NOT NULL,                                                                                                                        
                major           varchar(20)     NOT NULL,                                                                                                                        
                salary          varchar(20)     NOT NULL,                                                                                                                        
                industry        varchar(50)     NOT NULL,
                `condition`     varchar(20)     NOT NULL,
                job_features    varchar(100)    NOT NULL,
                comp_features   varchar(100)    NOT NULL,
                link            varchar(50)     NOT NULL,
                size            varchar(20)         NULL, 
                address         varchar(100)        NULL,
                `require`       varchar(2000)       NULL,
                location        varchar(50)         NULL);''')


re_tag = re.compile(u'con_list_item.*default_list')


for url in urls:
    options = webdriver.FirefoxOptions()
    options.set_headless()
    driver = webdriver.Firefox(firefox_options=options)
    driver.get(url)
    soup = bs(driver.page_source, 'lxml')
    page_num = soup.find(class_='span totalNum').string
    #for term in range(2):
    for term in range(int(page_num)):
        soup = bs(driver.page_source, 'lxml')
        tags = soup.find_all('li', class_=re_tag)
        for tag in tags:
            city = soup.find('a', class_='current_city current').string
            salary = tag['data-salary']
            company = tag['data-company']
            pos_name = tag['data-positionname']
            experience = tag.find('div', class_='li_b_l').contents[4].strip().split(' / ')[0]
            major = tag.find('div', class_='li_b_l').contents[4].strip().split(' / ')[1]
            industry = tag.find('div', class_='industry').string.strip().split(' / ')[0]
            condition = tag.find('div', class_='industry').string.strip().split(' / ')[1]
            job_features = ','.join(list(tag.find('div', class_='list_item_bot').find_all('div')[0].stripped_strings))
            comp_features = tag.find('div', class_='list_item_bot').find_all('div')[1].string.strip('“”')
            link = tag.find('a', class_='position_link')['href']
            
            #然后存入数据库
            cursor.execute('''INSERT INTO 
            job_info(pos_name, city, company, experience, major, salary, industry,
            `condition`, job_features, comp_features, link)
            VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            '''%(repr(pos_name), repr(city), repr(company), repr(experience), repr(major), repr(salary), repr(industry),
            repr(condition), repr(job_features), repr(comp_features), repr(link)))
            
            print(company)
    
        print('sleep')
        driver.find_element_by_xpath("//ul[@id='order']/li/div[4]/div[2]").click()
        time.sleep(5)  #如果没有加载完全，现在获取html还是之前网的，我家网有毒，所以宁愿多等一下
        print('-----------------------------------------------')
    driver.close()
            

conn.commit()

#cursor.execute('select link from job_info limit 5')
cursor.execute('select link from job_info where address is null')
pos_urls = cursor.fetchall()




for url in pos_urls:
    #driver = webdriver.Firefox()
    options = webdriver.FirefoxOptions()
    options.set_headless()
    driver = webdriver.Firefox(firefox_options=options)
    driver.get(url[0])
    soup = bs(driver.page_source, 'lxml')
    size = soup.find('i',class_='icon-glyph-figure').next_sibling.strip()
    address = ''.join(list(soup.find('div', class_='work_addr').stripped_strings)[:-1]).replace('-', '').replace(' ','')
    require = ''.join(list(soup.find('dd', class_='job_bt').div.stripped_strings))
    cursor.execute('''UPDATE job_info SET size=%s, address=%s, `require`=%s WHERE link=%s
                   '''%(repr(size), repr(address), repr(require), repr(url[0])))
    
    print(size, '\n', address, '\n', require, '\n\n')
    conn.commit()
    #time.sleep(2)
    driver.close()
#conn.commit()
#driver.close()


cursor.execute('select address from job_info')

address = cursor.fetchall()

for addr in address:
    url = 'https://restapi.amap.com/v3/geocode/geo?key=0d1ccccbc8a759ee19353a180c21723c&address=%s'%addr[0]
    r = requests.get(url)
    location = json.loads(r.content.decode('utf-8'))['geocodes'][0]['location']
    cursor.execute('UPDATE job_info SET location=%s where address=%s'%(repr(location), repr(addr[0])))
    print(location, addr[0])
    time.sleep(2)

conn.commit()

cursor.close()
conn.close()
